### Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [2]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [3]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [4]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [5]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

In [6]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-65    3.5   63     0.67   0        Ruim     1.00      
-34    2.8   30     0.38   1        Bom      0.99      
-62    7.2   37     0.81   0        Ruim     1.00      
-52    1.8   9      0.15   1        Bom      1.00      
-31    2.1   33     0.11   1        Bom      1.00      
-61    3.5   21     0.01   1        Bom      1.00      
-45    3.3   19     0.20   1        Bom      1.00      
-64    7.7   71     0.54   0        Ruim     1.00      
-53    3.3   40     0.32   1        Bom      0.93      
-39    2.0   10     0.04   1        Bom      1.00      
-63    2.7   3      0.39   1        Bom      0.98      
-87    3.5   50     0.54   0        Ruim     1.00      
-56    7.2   37     0.37   0        Ruim     0.93      
-89    8.3   83     0.59   0        Ruim     1.00      
-55    4.5   91     0.47   0        Ruim     1.00      
-32    2.9   21     0.15   1        Bom      1.0

### Save

In [7]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
